In [59]:
import pandas as pd
import numpy as np
import sys, os

# Pull in Observed Solar Radiation Data

In [60]:
jan01_df = pd.read_csv('data/aftp.cmdl.noaa.gov/data/radiation/solrad/hnx/2017/hnx17001.dat', sep='\s+', header=None, skiprows=2)
jan01_df = jan01_df.rename({0: "year", 1: "jday", 2: "month", 3: "day", 4:"hour", 5:"min",
                           6:"dt", 7:"zen", 8:"dw_psp", 9:"direct", 10:"diffuse", 11:"uvb",
                           12: "uvb_temp", 13:"qc_dwpsp", 14:"qc_direct", 15:"qc_diffuse",
                           16:"qc_uvb", 17:"qc_uvb_temp", 18:"std_dw_psp", 19:"std_direct",
                           20:"std_diffuse", 21:"std_uvb"}, axis='columns')
jan01_df.head()

,year,jday,month,day,hour,min,dt,zen,dw_psp,direct,...,uvb_temp,qc_dwpsp,qc_direct,qc_diffuse,qc_uvb,qc_uvb_temp,std_dw_psp,std_direct,std_diffuse,std_uvb
0,2017,1,1,1,0,0,0.000,81.48,18.1,0,...,21.5,0,1.7,0,43.3,0,0.865,0.242,0.885,0.000
1,2017,1,1,1,0,1,0.017,81.64,20.9,0,...,24.3,0,1.8,0,43.3,0,0.741,0.242,0.774,0.041
2,2017,1,1,1,0,2,0.033,81.80,23.1,0,...,26.6,0,1.9,0,43.3,0,0.494,0.000,0.442,0.000
3,2017,1,1,1,0,3,0.050,81.96,23.8,0,...,27.5,0,1.9,0,43.3,0,0.000,0.000,0.111,0.000
4,2017,1,1,1,0,4,0.067,82.12,23.3,0,...,27.1,0,1.8,0,43.3,0,0.371,0.000,0.332,0.041


In [61]:
jan01_hour_df = jan01_df.groupby(by="hour").sum()
jan01_hour_df

,year,jday,month,day,min,dt,zen,dw_psp,direct,diffuse,...,uvb_temp,qc_dwpsp,qc_direct,qc_diffuse,qc_uvb,qc_uvb_temp,std_dw_psp,std_direct,std_diffuse,std_uvb
hour,,,,,,,,,,,,,,,,,,,,,
0,121020,60,60,60,1770,29.5,5177.30,177.4,0,7.8,...,410.1,0,32.7,0,2598.0,0,11.495,2.299,11.395,0.688
1,121020,60,60,60,1770,89.5,5832.36,-241.3,0,16.4,...,-33.4,0,0.0,0,2598.0,0,3.095,2.662,5.094,0.000
2,121020,60,60,60,1770,149.5,6515.27,-185.9,0,9.5,...,-30.5,0,0.0,0,2598.0,0,3.095,2.420,5.314,0.000
3,121020,60,60,60,1770,209.5,7224.63,-211.3,0,3.9,...,-41.7,0,0.0,0,2598.0,0,2.968,1.452,1.220,0.000
4,121020,60,60,60,1770,269.5,7946.99,-204.6,0,13.5,...,-24.8,0,0.0,0,2598.0,0,2.600,3.630,1.440,0.000
5,121020,60,60,60,1770,329.5,8665.94,-236.8,0,4.7,...,-24.8,0,0.0,0,2598.0,0,3.340,2.904,1.220,0.000
6,121020,60,60,60,1770,389.5,9347.47,-233.2,0,46.5,...,-39.3,0,0.0,0,2598.0,0,3.711,0.968,1.994,0.000
7,121020,60,60,60,1770,449.5,9870.09,-199.3,0,67.1,...,-24.0,0,0.0,0,2598.0,0,1.982,1.815,0.111,0.000
8,121020,60,60,60,1770,509.5,9902.77,-272.5,0,2.7,...,-35.8,0,0.0,0,2598.0,0,1.486,2.178,2.327,0.000
